# Project Title: book club app

***Description***

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 

You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

## Goals of the study: Use data to generate a value proposition for books

## Connect to the Data Base

In [2]:
# import libraries
!python -m pip install psycopg2
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Study the tables

In [3]:
query = ''' SELECT *
            FROM books
        '''

In [4]:
books = pd.io.sql.read_sql(query, con = engine)
print(books.head())

   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  


In [5]:
query = ''' SELECT *
            FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine)
print(authors.head())

   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          5        Alan Moore/David   Lloyd


In [6]:
query = ''' SELECT *
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)
print(publishers.head())

   publisher_id                          publisher
0             1                                Ace
1             2                           Ace Book
2             3                          Ace Books
3             4                      Ace Hardcover
4             5  Addison Wesley Publishing Company


In [7]:
query = ''' SELECT *
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)
print(ratings.head())

   rating_id  book_id       username  rating
0          1        1     ryanfranco       4
1          2        1  grantpatricia       2
2          3        1   brandtandrea       5
3          4        2       lorichen       3
4          5        2    mariokeller       2


In [8]:
query = ''' SELECT *
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine)
print(reviews.head())

   review_id  book_id       username  \
0          1        1   brandtandrea   
1          2        1     ryanfranco   
2          3        2       lorichen   
3          4        3  johnsonamanda   
4          5        3    scotttamara   

                                                text  
0  Mention society tell send professor analysis. ...  
1  Foot glass pretty audience hit themselves. Amo...  
2  Listen treat keep worry. Miss husband tax but ...  
3  Finally month interesting blue could nature cu...  
4  Nation purpose heavy give wait song will. List...  


## Tasks

### Find the number of books released after January 1, 2000.

In [9]:
query = ''' SELECT count(book_id) num_books
            FROM books
            WHERE EXTRACT(YEAR FROM CAST(publication_date AS date)) > 2000
                or (EXTRACT(YEAR FROM CAST(publication_date AS date)) = 2000 
                and EXTRACT(DOY FROM CAST(publication_date AS date))!= 1)
        '''
library_df = pd.io.sql.read_sql(query, con = engine)
print(library_df)

   num_books
0        819


#### conclusions

- 819 books were released after January 1, 2000.

### Find the number of user reviews and the average rating for each book.

In [11]:
query = ''' SELECT rt.book_id id,
                   count(DISTINCT rw.review_id) num_rev,
                   avg(rt.rating) avg_rt
            FROM ratings as rt
            inner join reviews as rw on rt.book_id = rw.book_id 
            group by rt.book_id
            order by avg_rt desc
        '''
library_df = pd.io.sql.read_sql(query, con = engine)
print(library_df)

      id  num_rev  avg_rt
0    972        2    5.00
1    513        2    5.00
2    901        2    5.00
3    136        2    5.00
4     86        2    5.00
..   ...      ...     ...
989  915        3    2.25
990  371        2    2.00
991  316        2    2.00
992  202        3    2.00
993  303        2    1.50

[994 rows x 3 columns]


### Identify the publisher that has released the greatest number of books with more than 50 pages

In [12]:
query = ''' SELECT publisher
            FROM publishers
            where publisher_id = (SELECT bk.publisher_id   
            FROM books as bk
            inner join publishers as pub on bk.publisher_id = pub.publisher_id
            where bk.num_pages > 50
            group by bk.publisher_id
            order by count(bk.book_id) desc
            limit 1)
        '''
library_df = pd.io.sql.read_sql(query, con = engine)
print(library_df)

       publisher
0  Penguin Books


#### conclusions

- The publisher that has released the greatest number of books with more than 50 pages, is Penguin Books.

### Identify the author with the highest average book rating (look only at books with at least 50 ratings).

In [13]:
query = ''' SELECT author
            FROM authors
            where author_id = (SELECT bk.author_id author
                               FROM books as bk
                               inner join ratings as rt on bk.book_id = rt.book_id
                               WHERE bk.book_id IN (SELECT bk.book_id bk_id
                                                    FROM books as bk
                                                    inner join ratings as rt on bk.book_id = rt.book_id 
                                                    group by bk.book_id
                                                    having count(rating_id) >= 50)
                               group by bk.author_id
                               order by avg(rt.rating) desc
                               limit 1)
        '''
library_df = pd.io.sql.read_sql(query, con = engine)
print(library_df)

                       author
0  J.K. Rowling/Mary GrandPré


#### conclusions

- The author with the highest average book rating (among authors with books, with at least 50 ratings), is the duo J.K. Rowling/Mary GrandPré (J.K. Rowling is the author, while Mary GrandPré is the illustrator of her books).

### Find the average number of text reviews among users who rated more than 50 books. 

In [14]:
query = ''' SELECT AVG(tex_rw)
            FROM (SELECT username uname,
                  COUNT(text) tex_rw
                  FROM reviews as rw 
                  WHERE username IN (SELECT username uname               
                                     FROM ratings as rt
                                     GROUP BY username
                                     HAVING count(rating_id) > 50)
                  GROUP BY username) AS r50t                   
        '''
library_df = pd.io.sql.read_sql(query, con = engine)
print(library_df)

         avg
0  24.333333


#### conclusions

- The average number of text reviews among users who rated more than 50 books, is about 24 (24.333) text reviews.

## Overall conclusions

- 819 books were released after January 1, 2000.

- The publisher that has released the greatest number of books with more than 50 pages, is Penguin Books.

- The author with the highest average book rating (among authors with books, with at least 50 ratings), is the duo J.K. Rowling/Mary GrandPré (J.K. Rowling is the author, while Mary GrandPré is the illustrator of her books).

- The average number of text reviews among users who rated more than 50 books, is about 24 (24.333) text reviews.